In [ ]:
#run this first time for creating csv files and making coulmns
import csv
with open("road1.csv", 'wt') as write_obj:
    csv_writer = csv.writer(write_obj)
    arr=["CLASS","TIME","DATE"]
    csv_writer.writerow(arr)
write_obj.close()

with open("road2.csv", 'wt') as write_obj:
    csv_writer = csv.writer(write_obj)
    arr=["CLASS","TIME","DATE"]
    csv_writer.writerow(arr)
write_obj.close()

with open("road3.csv", 'wt') as write_obj:
    csv_writer = csv.writer(write_obj)
    arr=["CLASS","TIME","DATE"]
    csv_writer.writerow(arr)
write_obj.close()

with open("road4.csv", 'wt') as write_obj:
    csv_writer = csv.writer(write_obj)
    arr=["CLASS","TIME","DATE"]
    csv_writer.writerow(arr)
write_obj.close()




import cv2 as cv
import numpy as np
from datetime import datetime
import csv
 
cap1 = cv.VideoCapture("testing1.mp4")
cap2 = cv.VideoCapture("testing2.mp4")
cap3 = cv.VideoCapture("testing3.mp4")
cap4 = cv.VideoCapture("testing4.mp4")
whT = 320
confThreshold =0.5
nmsThreshold= 0.2
 
#### LOAD MODEL
## Coco Names
classesFile = "coco.names"
classNames = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
#print(classNames)
## Model Files
modelConfiguration = "yolov3-320.cfg"
modelWeights = "yolov3-320.weights"
net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)
 

    
    
    
def findObjects(outputs,img,cur,which_road):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []
    cts=[0,0,0,0,0]#count of vehicles for each class [bike,bus,car,truck]
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]*wT) , int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2) , int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))
                
                
 
    indices = cv.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)
    
    for i in indices:
        i = i[0]
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        # print(x,y,w,h)
        cv.rectangle(img, (x, y), (x+w,y+h), (255, 0 , 255), 2)
        bar_bar=classNames[classIds[i]]#bar_bar contains the class of the vehicle
        cv.putText(img,f'{bar_bar.upper()} {int(confs[i]*100)}%',
                  (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        
        if(bar_bar=="bicycle"):
            cts[0]+=1#incrementing bicycle count 
        elif(bar_bar=="motorbike"):
            cts[1]+=1#incrementing moterbike count
        elif(bar_bar=="bus"):
            cts[2]+=1#incrementing moterbike count
        elif(bar_bar=="car"):
            cts[3]+=1#incrementing moterbike count 
        elif(bar_bar=="truck"):
            cts[4]+=1#incrementing moterbike count
        row_contents = [bar_bar.upper(),cur.time(),cur.date()]
        # Append a list as new line to an old csv file
        if(which_road==1):
            append_list_as_row('road1.csv', row_contents)
        if(which_road==2):
            append_list_as_row('road2.csv', row_contents)
        if(which_road==3):
            append_list_as_row('road3.csv', row_contents)
        if(which_road==4):
            append_list_as_row('road4.csv', row_contents)
    return cts

def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = csv.writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)
    write_obj.close()

    
    
    
ptr=0#to loop every 4 second
while True:
    cur = datetime.now() #csv file me store karne ke liye 
    #print(cur)
    now = cur.time() # time object
    
    start=now.replace(hour=7, minute=0, second=0, microsecond=0)
    end=now.replace(hour=23, minute=0, second=0, microsecond=0)
    #print("start - {} and end - {}".format(start,end))

    if(now > start and now < end):
        temp = str(now).split(":")
        #print("inside1")
        #print(temp)
        #print(len(temp))
        if(ptr!=int(float(temp[2])) and int(float(temp[2]))%4==0 or float(temp[2])==0 ):# as considering one cycle as 4 seconds and 
                                                                    #at every cycle calculate green signal time    
                                                                    #as testing video files are small
                    
    
            #for displaying here we will use frames per 4 second insted of 4 minutes alright
                
                
            #print("inside2")
            ptr=int(float(temp[2]))
            
            legendary=[]#oo yeah fkin traffic management 4 element 4 road vehicle count(list of[list of[5 element class 
                                                                                                        #wise count]])
             
            success1, img1 = cap1.read()
            success2, img2 = cap2.read()
            success3, img3 = cap3.read()
            success4, img4 = cap4.read()

            blob1 = cv.dnn.blobFromImage(img1, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)
            blob2 = cv.dnn.blobFromImage(img2, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)
            blob3 = cv.dnn.blobFromImage(img3, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)
            blob4 = cv.dnn.blobFromImage(img4, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)


            layersNames = net.getLayerNames()
            outputNames = [(layersNames[i[0] - 1]) for i in net.getUnconnectedOutLayers()]

            
            #for first road
            net.setInput(blob1)
            outputs1 = net.forward(outputNames)
            cts=findObjects(outputs1,img1,cur,1)     #cts contains the count of each vechcle in the gicen image
            legendary.append(cts)
            print(legendary[0])
            print("vehivle count in road A- ",sum(legendary[0])) 
            cv.imshow('Image', img1)
            
            
            #for second road
            net.setInput(blob2)
            outputs2 = net.forward(outputNames)
            cst=findObjects(outputs2,img2,cur,2)
            legendary.append(cst)
            print(legendary[1])
            print("vehivle count in road B- ",sum(legendary[1]))
            cv.imshow('Image2', img2)

            #for 3rd road
            net.setInput(blob3)
            outputs3 = net.forward(outputNames)
            cst=findObjects(outputs3,img3,cur,3)
            legendary.append(cst)
            print(legendary[2])
            print("vehivle count in road C- ",sum(legendary[2]))
            cv.imshow('Image3', img3)

            #for 4th road
            net.setInput(blob4)
            outputs4 = net.forward(outputNames)
            cst=findObjects(outputs4,img4,cur,4)
            legendary.append(cst)
            print(legendary[3])
            print("vehivle count in road D- ",sum(legendary[3]))
            cv.imshow('Image4', img4)
            
            
            if cv.waitKey(1) & 0xFF == ord('q'):
                
                break
            
            
            
            
            
            #DAm jo pehele claim kiya(code dikhaya tha) tha wo kar ke dikhaya leeh bhendi
            
            vclaneA=legendary[0][0]+legendary[0][1]+legendary[0][2]+legendary[0][3]+legendary[0][4]
            vclaneB=legendary[1][0]+legendary[1][1]+legendary[1][2]+legendary[1][3]+legendary[1][4]
            vclaneC=legendary[2][0]+legendary[2][1]+legendary[2][2]+legendary[2][3]+legendary[2][4]
            vclaneD=legendary[3][0]+legendary[3][1]+legendary[3][2]+legendary[3][3]+legendary[3][4]
            total_time=240 #4 mins,1 mins each or whatever is the default time
            total_vehicals=vclaneA+vclaneB+vclaneC+vclaneD
            
            #if(Ambulance_detected()):#Ambulance_detected returns lane in which ambulance is and 0 if not detected
            #lane_in_amb=Ambulance_detected()
            #while(Ambulance_detected()):
            #greensignaltime_of_detected_lane=240

            greensignaltimeA=(vclaneA/total_vehicals)*total_time
            greensignaltimeB=(vclaneB/total_vehicals)*total_time
            greensignaltimeC=(vclaneC/total_vehicals)*total_time
            greensignaltimeD=(vclaneD/total_vehicals)*total_time
            print("greensignaltimeA - ",greensignaltimeA)
            print("greensignaltimeB - ",greensignaltimeB)
            print("greensignaltimeC - ",greensignaltimeC)
            print("greensignaltimeD - ",greensignaltimeD)
            print("------------------------------------")
            
            
            #set signal time 
            #wait 4 mins
cap1.release()
cap2.release()
cap3.release()
cap4.release()
cv.destroyAllWindows()
            
    
    
    
    

[0, 0, 0, 14, 1]
vehivle count in road A-  15
[0, 0, 0, 17, 0]
vehivle count in road B-  17
[0, 0, 0, 17, 0]
vehivle count in road C-  17
[0, 0, 0, 10, 0]
vehivle count in road D-  10
greensignaltimeA -  61.01694915254237
greensignaltimeB -  69.15254237288136
greensignaltimeC -  69.15254237288136
greensignaltimeD -  40.67796610169491
------------------------------------
[0, 0, 0, 14, 1]
vehivle count in road A-  15
[0, 0, 0, 18, 0]
vehivle count in road B-  18
[0, 0, 0, 17, 0]
vehivle count in road C-  17
[0, 0, 0, 10, 0]
vehivle count in road D-  10
greensignaltimeA -  60.0
greensignaltimeB -  72.0
greensignaltimeC -  68.0
greensignaltimeD -  40.0
------------------------------------
[0, 0, 0, 13, 1]
vehivle count in road A-  14
[0, 0, 0, 17, 0]
vehivle count in road B-  17
[0, 0, 0, 17, 0]
vehivle count in road C-  17
[0, 0, 0, 10, 0]
vehivle count in road D-  10
greensignaltimeA -  57.931034482758626
greensignaltimeB -  70.34482758620689
greensignaltimeC -  70.34482758620689
greensi

[0, 0, 0, 9, 0]
vehivle count in road D-  9
greensignaltimeA -  56.72727272727273
greensignaltimeB -  74.18181818181819
greensignaltimeC -  69.81818181818181
greensignaltimeD -  39.27272727272727
------------------------------------
[0, 0, 0, 13, 0]
vehivle count in road A-  13
[0, 0, 0, 16, 1]
vehivle count in road B-  17
[0, 0, 0, 14, 0]
vehivle count in road C-  14
[0, 0, 0, 8, 0]
vehivle count in road D-  8
greensignaltimeA -  60.0
greensignaltimeB -  78.46153846153847
greensignaltimeC -  64.61538461538461
greensignaltimeD -  36.92307692307693
------------------------------------
[0, 0, 0, 13, 0]
vehivle count in road A-  13
[0, 0, 0, 16, 1]
vehivle count in road B-  17
[0, 0, 0, 16, 0]
vehivle count in road C-  16
[0, 0, 0, 8, 0]
vehivle count in road D-  8
greensignaltimeA -  57.77777777777777
greensignaltimeB -  75.55555555555556
greensignaltimeC -  71.11111111111111
greensignaltimeD -  35.55555555555556
------------------------------------
[0, 0, 0, 13, 0]
vehivle count in road

[0, 0, 0, 10, 0]
vehivle count in road A-  10
[0, 0, 0, 16, 1]
vehivle count in road B-  17
[0, 0, 0, 11, 0]
vehivle count in road C-  11
[0, 0, 0, 9, 0]
vehivle count in road D-  9
greensignaltimeA -  51.06382978723404
greensignaltimeB -  86.80851063829788
greensignaltimeC -  56.170212765957444
greensignaltimeD -  45.95744680851064
------------------------------------
[0, 0, 0, 12, 0]
vehivle count in road A-  12
[0, 0, 0, 16, 1]
vehivle count in road B-  17
[0, 0, 0, 11, 0]
vehivle count in road C-  11
[0, 0, 0, 9, 0]
vehivle count in road D-  9
greensignaltimeA -  58.775510204081634
greensignaltimeB -  83.26530612244898
greensignaltimeC -  53.87755102040816
greensignaltimeD -  44.08163265306123
------------------------------------
[0, 0, 0, 12, 0]
vehivle count in road A-  12
[0, 0, 0, 16, 1]
vehivle count in road B-  17
[0, 0, 0, 12, 0]
vehivle count in road C-  12
[0, 0, 0, 9, 0]
vehivle count in road D-  9
greensignaltimeA -  57.599999999999994
greensignaltimeB -  81.600000000000

[0, 1, 0, 11, 1]
vehivle count in road A-  13
[0, 0, 0, 15, 0]
vehivle count in road B-  15
[0, 0, 0, 17, 0]
vehivle count in road C-  17
[0, 0, 0, 9, 0]
vehivle count in road D-  9
greensignaltimeA -  57.77777777777777
greensignaltimeB -  66.66666666666667
greensignaltimeC -  75.55555555555556
greensignaltimeD -  40.0
------------------------------------
[0, 1, 0, 12, 1]
vehivle count in road A-  14
[0, 0, 0, 15, 0]
vehivle count in road B-  15
[0, 0, 0, 17, 0]
vehivle count in road C-  17
[0, 0, 0, 9, 0]
vehivle count in road D-  9
greensignaltimeA -  61.090909090909086
greensignaltimeB -  65.45454545454545
greensignaltimeC -  74.18181818181819
greensignaltimeD -  39.27272727272727
------------------------------------
[0, 1, 0, 13, 0]
vehivle count in road A-  14
[0, 0, 0, 15, 0]
vehivle count in road B-  15
[0, 0, 0, 17, 0]
vehivle count in road C-  17
[0, 0, 0, 9, 0]
vehivle count in road D-  9
greensignaltimeA -  61.090909090909086
greensignaltimeB -  65.45454545454545
greensignal

[0, 1, 0, 11, 0]
vehivle count in road A-  12
[0, 0, 0, 15, 1]
vehivle count in road B-  16
[0, 0, 0, 16, 0]
vehivle count in road C-  16
[0, 0, 0, 10, 0]
vehivle count in road D-  10
greensignaltimeA -  53.33333333333333
greensignaltimeB -  71.11111111111111
greensignaltimeC -  71.11111111111111
greensignaltimeD -  44.44444444444444
------------------------------------
[0, 0, 0, 9, 0]
vehivle count in road A-  9
[0, 0, 0, 15, 0]
vehivle count in road B-  15
[0, 0, 0, 16, 0]
vehivle count in road C-  16
[0, 0, 0, 10, 0]
vehivle count in road D-  10
greensignaltimeA -  43.199999999999996
greensignaltimeB -  72.0
greensignaltimeC -  76.8
greensignaltimeD -  48.0
------------------------------------
[0, 1, 0, 8, 0]
vehivle count in road A-  9
[0, 0, 0, 16, 0]
vehivle count in road B-  16
[0, 0, 0, 16, 0]
vehivle count in road C-  16
[0, 0, 0, 10, 0]
vehivle count in road D-  10
greensignaltimeA -  42.352941176470594
greensignaltimeB -  75.29411764705883
greensignaltimeC -  75.294117647058

...

[0, 0, 0, 13, 0]
vehivle count in road A-  13
[0, 0, 0, 12, 1]
vehivle count in road B-  13
[0, 0, 0, 17, 0]
vehivle count in road C-  17
[0, 0, 0, 10, 0]
vehivle count in road D-  10
greensignaltimeA -  58.867924528301884
greensignaltimeB -  58.867924528301884
greensignaltimeC -  76.98113207547169
greensignaltimeD -  45.28301886792453
------------------------------------
[0, 0, 0, 13, 0]
vehivle count in road A-  13
[0, 0, 0, 12, 1]
vehivle count in road B-  13
[0, 0, 0, 17, 0]
vehivle count in road C-  17
[0, 0, 0, 10, 0]
vehivle count in road D-  10
greensignaltimeA -  58.867924528301884
greensignaltimeB -  58.867924528301884
greensignaltimeC -  76.98113207547169
greensignaltimeD -  45.28301886792453
------------------------------------
[0, 0, 0, 13, 0]
vehivle count in road A-  13
[0, 0, 0, 12, 1]
vehivle count in road B-  13
[0, 0, 0, 15, 0]
vehivle count in road C-  15
[0, 0, 0, 10, 0]
vehivle count in road D-  10
greensignaltimeA -  61.17647058823529
greensignaltimeB -  61.17647

[0, 0, 0, 14, 1]
vehivle count in road B-  15
[0, 1, 0, 17, 0]
vehivle count in road C-  18
[0, 0, 0, 13, 1]
vehivle count in road D-  14
greensignaltimeA -  58.064516129032256
greensignaltimeB -  58.064516129032256
greensignaltimeC -  69.67741935483872
greensignaltimeD -  54.193548387096776
------------------------------------
[0, 0, 0, 15, 0]
vehivle count in road A-  15
[0, 0, 0, 14, 1]
vehivle count in road B-  15
[0, 0, 0, 19, 0]
vehivle count in road C-  19
[0, 0, 0, 13, 0]
vehivle count in road D-  13
greensignaltimeA -  58.064516129032256
greensignaltimeB -  58.064516129032256
greensignaltimeC -  73.54838709677419
greensignaltimeD -  50.322580645161295
------------------------------------
[0, 0, 0, 15, 0]
vehivle count in road A-  15
[0, 0, 0, 14, 1]
vehivle count in road B-  15
[0, 0, 0, 19, 0]
vehivle count in road C-  19
[0, 0, 0, 13, 0]
vehivle count in road D-  13
greensignaltimeA -  58.064516129032256
greensignaltimeB -  58.064516129032256
greensignaltimeC -  73.548387096

[0, 0, 0, 16, 0]
vehivle count in road A-  16
[0, 0, 0, 12, 2]
vehivle count in road B-  14
[0, 1, 0, 11, 0]
vehivle count in road C-  12
[0, 0, 0, 14, 1]
vehivle count in road D-  15
greensignaltimeA -  67.36842105263158
greensignaltimeB -  58.94736842105263
greensignaltimeC -  50.526315789473685
greensignaltimeD -  63.1578947368421
------------------------------------
[0, 0, 0, 19, 0]
vehivle count in road A-  19
[0, 0, 0, 11, 2]
vehivle count in road B-  13
[0, 1, 0, 11, 0]
vehivle count in road C-  12
[0, 0, 0, 14, 1]
vehivle count in road D-  15
greensignaltimeA -  77.28813559322033
greensignaltimeB -  52.88135593220339
greensignaltimeC -  48.8135593220339
greensignaltimeD -  61.01694915254237
------------------------------------
[0, 0, 0, 19, 0]
vehivle count in road A-  19
[0, 0, 0, 12, 2]
vehivle count in road B-  14
[0, 1, 0, 12, 0]
vehivle count in road C-  13
[0, 0, 0, 14, 0]
vehivle count in road D-  14
greensignaltimeA -  76.0
greensignaltimeB -  56.0
greensignaltimeC -  5

[0, 0, 0, 16, 0]
vehivle count in road D-  16
greensignaltimeA -  58.94736842105263
greensignaltimeB -  54.73684210526316
greensignaltimeC -  58.94736842105263
greensignaltimeD -  67.36842105263158
------------------------------------
[0, 0, 0, 14, 0]
vehivle count in road A-  14
[0, 0, 0, 13, 1]
vehivle count in road B-  14
[0, 0, 0, 13, 0]
vehivle count in road C-  13
[0, 0, 0, 16, 0]
vehivle count in road D-  16
greensignaltimeA -  58.94736842105263
greensignaltimeB -  58.94736842105263
greensignaltimeC -  54.73684210526316
greensignaltimeD -  67.36842105263158
------------------------------------
[0, 0, 0, 14, 0]
vehivle count in road A-  14
[0, 0, 0, 13, 1]
vehivle count in road B-  14
[0, 0, 0, 15, 0]
vehivle count in road C-  15
[0, 0, 0, 16, 0]
vehivle count in road D-  16
greensignaltimeA -  56.949152542372886
greensignaltimeB -  56.949152542372886
greensignaltimeC -  61.01694915254237
greensignaltimeD -  65.08474576271186
------------------------------------
[0, 0, 0, 12, 0]


[0, 0, 0, 13, 0]
vehivle count in road A-  13
[0, 0, 0, 12, 1]
vehivle count in road B-  13
[0, 0, 0, 11, 0]
vehivle count in road C-  11
[0, 0, 0, 13, 1]
vehivle count in road D-  14
greensignaltimeA -  61.17647058823529
greensignaltimeB -  61.17647058823529
greensignaltimeC -  51.76470588235294
greensignaltimeD -  65.88235294117648
------------------------------------
[0, 0, 0, 13, 0]
vehivle count in road A-  13
[0, 0, 0, 12, 1]
vehivle count in road B-  13
[0, 0, 0, 10, 0]
vehivle count in road C-  10
[0, 0, 0, 13, 1]
vehivle count in road D-  14
greensignaltimeA -  62.400000000000006
greensignaltimeB -  62.400000000000006
greensignaltimeC -  48.0
greensignaltimeD -  67.2
------------------------------------
[0, 0, 0, 13, 0]
vehivle count in road A-  13
[0, 0, 0, 11, 1]
vehivle count in road B-  12
[0, 0, 0, 11, 0]
vehivle count in road C-  11
[0, 0, 0, 13, 1]
vehivle count in road D-  14
greensignaltimeA -  62.400000000000006
greensignaltimeB -  57.599999999999994
greensignaltimeC